In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import zipfile
# import shutil

# %cd /content/drive/MyDrive/2023 Samsung AI Challenge : Camera-Invariant Domain Adaptation/open

# !unzip -qq "/content/drive/MyDrive/2023 Samsung AI Challenge : Camera-Invariant Domain Adaptation/open.zip"

In [ ]:
import os

test_image_path = "/content/drive/MyDrive/2023 Samsung AI Challenge : Camera-Invariant Domain Adaptation/open/test_image"
train_source_gt_path = "/content/drive/MyDrive/2023 Samsung AI Challenge : Camera-Invariant Domain Adaptation/open/train_source_gt"
train_source_image_path = "/content/drive/MyDrive/2023 Samsung AI Challenge : Camera-Invariant Domain Adaptation/open/train_source_image"
train_target_image_path = "/content/drive/MyDrive/2023 Samsung AI Challenge : Camera-Invariant Domain Adaptation/open/train_target_image"
val_source_gt_path = "/content/drive/MyDrive/2023 Samsung AI Challenge : Camera-Invariant Domain Adaptation/open/val_source_gt"
val_source_image_path = "/content/drive/MyDrive/2023 Samsung AI Challenge : Camera-Invariant Domain Adaptation/open/val_source_image"

file_list1 = os.listdir(test_image_path)
file_list2 = os.listdir(train_source_gt_path)
file_list3 = os.listdir(train_source_image_path)
file_list4 = os.listdir(train_target_image_path)
file_list5 = os.listdir(val_source_gt_path)
file_list6 = os.listdir(val_source_image_path)

file_count1 = len(file_list1)
file_count2 = len(file_list2)
file_count3 = len(file_list3)
file_count4 = len(file_list4)
file_count5 = len(file_list5)
file_count6 = len(file_list6)

print('test_image :', file_count1)
print('train_source_gt :', file_count2)
print('train_source_image :', file_count3)
print('train_target_image :', file_count4)
print('val_source_gt :', file_count5)
print('val_source_image :', file_count6)

# test_image         : 1898
# train_source_gt    : 2194
# train_source_image : 2194
# train_target_image : 2923
# val_source_gt      : 466
# val_source_image   : 466

test_image : 1898
train_source_gt : 2194
train_source_image : 2194
train_target_image : 2923
val_source_gt : 466
val_source_image : 466


## Import

In [ ]:
!pip install efficientnet-pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=e055e17591b1a1c1db7b89e7820153f8454f675d5fea562a6bba9209852d9407
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet-pytorch


In [ ]:
import cv2
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.optim import lr_scheduler
import torchvision.models.detection.mask_rcnn
import torchvision.models as models
from efficientnet_pytorch import EfficientNet

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# 현재 작업 디렉토리 확인
current_directory = os.getcwd()
print("Current working directory:", current_directory)

# 새로운 작업 디렉토리로 변경
new_directory = "/content/drive/MyDrive/2023 Samsung AI Challenge : Camera-Invariant Domain Adaptation/open"  # 새로운 디렉토리 경로를 지정
os.chdir(new_directory)

# 변경된 현재 작업 디렉토리 확인
current_directory = os.getcwd()
print("New current working directory:", current_directory)

Current working directory: /content/drive/MyDrive/2023 Samsung AI Challenge : Camera-Invariant Domain Adaptation/open
New current working directory: /content/drive/MyDrive/2023 Samsung AI Challenge : Camera-Invariant Domain Adaptation/open


## Utils

In [ ]:
# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

## Custom Dataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None, infer=False):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx, 1]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image

        mask_path = self.data.iloc[idx, 2]
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask[mask == 255] = 12 #배경을 픽셀값 12로 간주

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask

In [ ]:
# class CustomDataset(Dataset):
#     def __init__(self, csv_file, transform=None, infer=False):
#         self.data = pd.read_csv(csv_file)
#         self.transform = transform
#         self.infer = infer

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         img_path = self.data.iloc[idx, 1]
#         image = cv2.imread(img_path)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

#         if self.infer:
#             # 가우시안 필터를 적용합니다.
#             image = cv2.GaussianBlur(image, (5, 5), 0)  # (5, 5)는 커널 크기, 0은 표준 편차입니다.

#             if self.transform:
#                 image = self.transform(image=image)['image']
#             return image

#         mask_path = self.data.iloc[idx, 2]
#         mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
#         mask[mask == 255] = 12  # 배경을 픽셀값 12로 간주

#         if self.transform:
#             augmented = self.transform(image=image, mask=mask)
#             image = augmented['image']
#             mask = augmented['mask']

#         return image, mask

## Data Loader

In [ ]:
transform = A.Compose(
    [
        A.Resize(224, 224),
        A.Normalize(),
        ToTensorV2()
    ]
)

dataset = CustomDataset(csv_file='./train_source.csv', transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=4)

val_dataset = CustomDataset(csv_file='./val_source.csv', transform=transform)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=True, num_workers=4)

## Define Model

In [ ]:
# U-Net의 기본 구성 요소인 Double Convolution Block을 정의합니다.
def double_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True)
    )

# 간단한 U-Net 모델 정의
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        self.dconv_down1 = double_conv(3, 64)
        self.dconv_down2 = double_conv(64, 128)
        self.dconv_down3 = double_conv(128, 256)
        self.dconv_down4 = double_conv(256, 512)

        self.maxpool = nn.MaxPool2d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.dconv_up3 = double_conv(256 + 512, 256)
        self.dconv_up2 = double_conv(128 + 256, 128)
        self.dconv_up1 = double_conv(128 + 64, 64)

        self.conv_last = nn.Conv2d(64, 13, 1) # 12개 class + 1 background

    def forward(self, x):
        conv1 = self.dconv_down1(x)
        x = self.maxpool(conv1)

        conv2 = self.dconv_down2(x)
        x = self.maxpool(conv2)

        conv3 = self.dconv_down3(x)
        x = self.maxpool(conv3)

        x = self.dconv_down4(x)

        x = self.upsample(x)
        x = torch.cat([x, conv3], dim=1)

        x = self.dconv_up3(x)
        x = self.upsample(x)
        x = torch.cat([x, conv2], dim=1)

        x = self.dconv_up2(x)
        x = self.upsample(x)
        x = torch.cat([x, conv1], dim=1)

        x = self.dconv_up1(x)

        out = self.conv_last(x)

        return out

#	0.1466746583

In [ ]:
# SegNet의 인코더 부분 정의
class SegNetEncoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(SegNetEncoder, self).__init__()
        self.encode = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, return_indices=True)
        )

    def forward(self, x):
        x, indices = self.encode(x)
        return x, indices

# SegNet의 디코더 부분 정의
class SegNetDecoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(SegNetDecoder, self).__init__()
        self.decode = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x, indices):
        x = nn.functional.max_unpool2d(x, indices, kernel_size=2, stride=2)
        x = self.decode(x)
        return x

# 모델 아키텍처에서 indices 정보를 저장하도록 수정
class SegNet(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(SegNet, self).__init__()
        self.encoder1 = SegNetEncoder(in_channels, 64)
        self.encoder2 = SegNetEncoder(64, 128)
        self.encoder3 = SegNetEncoder(128, 256)
        self.encoder4 = SegNetEncoder(256, 512)

        self.decoder4 = SegNetDecoder(512, 256)
        self.decoder3 = SegNetDecoder(256, 128)
        self.decoder2 = SegNetDecoder(128, 64)
        self.decoder1 = SegNetDecoder(64, num_classes)

    def forward(self, x):
        x, indices1 = self.encoder1(x)
        x, indices2 = self.encoder2(x)
        x, indices3 = self.encoder3(x)
        x, indices4 = self.encoder4(x)

        x = self.decoder4(x, indices4)
        x = self.decoder3(x, indices3)
        x = self.decoder2(x, indices2)
        x = self.decoder1(x, indices1)

        return x

# 0.1249705711

In [ ]:
class FCN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(FCN, self).__init__()

        # Encoder
        self.encoder_conv1 = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, padding=100),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)
        )

        self.encoder_conv2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)
        )

        self.encoder_conv3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)
        )

        self.encoder_conv4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)
        )

        self.encoder_conv5 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, ceil_mode=True)
        )

        # Fully Convolutional Layers
        self.fully_conv1 = nn.Conv2d(512, 4096, kernel_size=7)
        self.fully_conv2 = nn.Conv2d(4096, 4096, kernel_size=1)

        # Score layers
        self.score_fr = nn.Conv2d(4096, num_classes, kernel_size=1)
        self.upscore = nn.ConvTranspose2d(num_classes, num_classes, kernel_size=64, stride=32, bias=False)

    def forward(self, x):
        # Encoder
        x1 = self.encoder_conv1(x)
        x2 = self.encoder_conv2(x1)
        x3 = self.encoder_conv3(x2)
        x4 = self.encoder_conv4(x3)
        x5 = self.encoder_conv5(x4)

        # Fully Convolutional Layers
        x = self.fully_conv1(x5)
        x = self.fully_conv2(x)

        # Score layers
        x = self.score_fr(x)

        # Upsample
        x = self.upscore(x)
        x = x[:, :, 64:64 + x.size(2), 64:64 + x.size(3)].contiguous()  # Crop the output to match the target size (224x224)

        return x

# 0.1251070643

In [ ]:
# class LightweightPSPNet(nn.Module):
#     def __init__(self, in_channels, num_classes):
#         super(LightweightPSPNet, self).__init__()

#         # Lightweight Backbone 네트워크 (예: ResNet-18)
#         self.backbone = models.resnet18(pretrained=True)
#         self.backbone.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
#         self.backbone.fc = nn.Identity()  # Fully-connected 레이어를 Identity로 대체하여 제거

#         # Pyramid Pooling Module 정의
#         self.ppm = PSPModule(512, 256, sizes=(1, 2, 3, 6))

#         # Classifier
#         self.final = nn.Sequential(
#             nn.Conv2d(512, num_classes, kernel_size=1)  # ResNet-18의 출력 채널 수는 512
#         )

#     def forward(self, x):
#         # Backbone 네트워크 수행
#         x = self.backbone.conv1(x)
#         x = self.backbone.bn1(x)
#         x = self.backbone.relu(x)
#         x = self.backbone.maxpool(x)
#         x1 = self.backbone.layer1(x)
#         x2 = self.backbone.layer2(x1)
#         x3 = self.backbone.layer3(x2)
#         x4 = self.backbone.layer4(x3)

#         # Pyramid Pooling Module 수행
#         x_ppm = self.ppm(x4)

#         # Upsampling 및 병합
#         x_ppm = F.interpolate(x_ppm, size=x3.size()[2:], mode='bilinear', align_corners=True)  # 크기를 x3와 일치하도록 조정
#         x = torch.cat([x3, x_ppm], dim=1)

#         # Classifier 수행
#         x = self.final(x)

#         # 마지막으로 224x224 크기로 다시 조정
#         x = F.interpolate(x, size=(224, 224), mode='bilinear', align_corners=True)

#         return x

# class PSPModule(nn.Module):
#     def __init__(self, in_channels, out_channels, sizes=(1, 2, 3, 6)):
#         super(PSPModule, self).__init__()

#         self.stages = nn.ModuleList([self._make_stage(in_channels, size) for size in sizes])
#         self.bottleneck = nn.Conv2d(in_channels + len(sizes) * out_channels, out_channels, kernel_size=3, padding=1)
#         self.relu = nn.ReLU()

#     def _make_stage(self, in_channels, size):
#         prior = nn.AdaptiveAvgPool2d(output_size=(size, size))
#         conv = nn.Conv2d(in_channels, 256, kernel_size=1, bias=False)
#         relu = nn.ReLU()
#         return nn.Sequential(prior, conv, relu)

#     def forward(self, x):
#         priors = [stage(x) for stage in self.stages] + [x]
#         priors = [F.interpolate(prior, size=x.shape[2:], mode='bilinear', align_corners=True) for prior in priors]
#         priors = torch.cat(priors, dim=1)
#         priors = self.bottleneck(priors)
#         return self.relu(priors)

# # 모델 초기화
# in_channels = 3  # 입력 이미지의 채널 수
# num_classes = 13  # 클래스 개수 설정 (배경 클래스 + 객체 클래스)
# model = LightweightPSPNet(in_channels, num_classes)

# # 입력 이미지 크기 설정 (예: 256x256)
# input_size = (256, 256)
# x = torch.randn(1, in_channels, *input_size)
# output = model(x)
# print("Output Shape:", output.shape)

# # resnet-18 ; 0.16314
# step_size=10에서 Epoch 20, Loss: 0.13495590402812196
# step_size=15에서 Epoch 20, Loss: 0.12928759333664092 / Epoch 30, Loss: 0.12090408715648927 -> 0.17987

In [ ]:
# class LightweightPSPNet(nn.Module):
#     def __init__(self, in_channels, num_classes):
#         super(LightweightPSPNet, self).__init__()

#         # EfficientNet B0 모델 불러오기
#         self.backbone = EfficientNet.from_pretrained('efficientnet-b0', in_channels=in_channels)

#         # Pyramid Pooling Module 정의
#         self.ppm = PSPModule(1280, 256, sizes=(1, 2, 3, 6))  # EfficientNet B0의 출력 채널 수는 1280

#         # Classifier
#         self.final = nn.Sequential(
#             nn.Conv2d(1536, num_classes, kernel_size=1)  # EfficientNet B0의 출력 채널 수는 1536
#         )

#     def forward(self, x):
#         # Backbone 네트워크 수행
#         x = self.backbone.extract_features(x)

#         # Pyramid Pooling Module 수행
#         x_ppm = self.ppm(x)

#         # Upsampling 및 병합
#         x_ppm = nn.functional.interpolate(x_ppm, size=x.size()[2:], mode='bilinear', align_corners=True)
#         x = torch.cat([x, x_ppm], dim=1)

#         # Classifier 수행
#         x = self.final(x)

#         # 마지막으로 224x224 크기로 다시 조정
#         x = nn.functional.interpolate(x, size=(224, 224), mode='bilinear', align_corners=True)

#         return x

# class PSPModule(nn.Module):
#     def __init__(self, in_channels, out_channels, sizes=(1, 2, 3, 6)):
#         super(PSPModule, self).__init__()

#         self.stages = nn.ModuleList([self._make_stage(in_channels, size) for size in sizes])
#         self.bottleneck = nn.Conv2d(in_channels + len(sizes) * out_channels, out_channels, kernel_size=3, padding=1)
#         self.relu = nn.ReLU()

#     def _make_stage(self, in_channels, size):
#         prior = nn.AdaptiveAvgPool2d(output_size=(size, size))
#         conv = nn.Conv2d(in_channels, 256, kernel_size=1, bias=False)
#         relu = nn.ReLU()
#         return nn.Sequential(prior, conv, relu)

#     def forward(self, x):
#         priors = [stage(x) for stage in self.stages] + [x]
#         priors = [nn.functional.interpolate(prior, size=x.shape[2:], mode='bilinear', align_corners=True) for prior in priors]
#         priors = torch.cat(priors, dim=1)
#         priors = self.bottleneck(priors)
#         return self.relu(priors)

# # 모델 초기화
# in_channels = 3  # 입력 이미지의 채널 수
# num_classes = 13  # 클래스 개수 설정 (배경 클래스 + 객체 클래스)
# model = LightweightPSPNet(in_channels, num_classes)

# # 입력 이미지 크기 설정 (예: 256x256)
# input_size = (256, 256)
# x = torch.randn(1, in_channels, *input_size)
# output = model(x)
# print("Output Shape:", output.shape)

# # efficientnet-b0 ; 0.2425057713
# # step_size=10 , runtime : 30m
# # Epoch 30
# # Train Loss: 0.20323605360328287, Train Accuracy: 92.18%
# # Validation Loss: 0.44996941884358727, Validation Accuracy: 87.96%

In [ ]:
class LightweightPSPNet(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(LightweightPSPNet, self).__init__()

        # EfficientNet B7 모델 불러오기
        self.backbone = EfficientNet.from_pretrained('efficientnet-b7', in_channels=in_channels)

        # Pyramid Pooling Module 정의
        self.ppm = PSPModule(2560, 256, sizes=(1, 2, 3, 6))  # EfficientNet B7의 출력 채널 수는 2560

        # Classifier
        self.final = nn.Sequential(
            nn.Conv2d(2816, num_classes, kernel_size=1)  # EfficientNet B7의 출력 채널 수는 2816
        )

    def forward(self, x):
        # Backbone 네트워크 수행
        x = self.backbone.extract_features(x)

        # Pyramid Pooling Module 수행
        x_ppm = self.ppm(x)

        # Upsampling 및 병합
        x_ppm = nn.functional.interpolate(x_ppm, size=x.size()[2:], mode='bilinear', align_corners=True)
        x = torch.cat([x, x_ppm], dim=1)

        # Classifier 수행
        x = self.final(x)

        # 마지막으로 224x224 크기로 다시 조정
        x = nn.functional.interpolate(x, size=(224, 224), mode='bilinear', align_corners=True)

        return x

class PSPModule(nn.Module):
    def __init__(self, in_channels, out_channels, sizes=(1, 2, 3, 6)):
        super(PSPModule, self).__init__()

        self.stages = nn.ModuleList([self._make_stage(in_channels, size) for size in sizes])
        self.bottleneck = nn.Conv2d(in_channels + len(sizes) * out_channels, out_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU()

    def _make_stage(self, in_channels, size):
        prior = nn.AdaptiveAvgPool2d(output_size=(size, size))
        conv = nn.Conv2d(in_channels, 256, kernel_size=1, bias=False)
        relu = nn.ReLU()
        return nn.Sequential(prior, conv, relu)

    def forward(self, x):
        priors = [stage(x) for stage in self.stages] + [x]
        priors = [nn.functional.interpolate(prior, size=x.shape[2:], mode='bilinear', align_corners=True) for prior in priors]
        priors = torch.cat(priors, dim=1)
        priors = self.bottleneck(priors)
        return self.relu(priors)

# 모델 초기화
in_channels = 3  # 입력 이미지의 채널 수
num_classes = 13  # 클래스 개수 설정 (배경 클래스 + 객체 클래스)
model = LightweightPSPNet(in_channels, num_classes)

# 입력 이미지 크기 설정 (예: 256x256)
input_size = (256, 256)
x = torch.randn(1, in_channels, *input_size)
output = model(x)
print("Output Shape:", output.shape)

# efficientnet-b7 / Validation Accuracy: 88.90%

Loaded pretrained weights for efficientnet-b7
Output Shape: torch.Size([1, 13, 224, 224])


## Model Train

In [ ]:
# 모델을 저장할 디렉토리 설정
save_dir = './saved_models'
# os.makedirs(save_dir, exist_ok=True) / 경로 생성

In [ ]:
# model 초기화
model = LightweightPSPNet(in_channels, num_classes).to(device)

# loss function과 optimizer 정의
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# step_size: 학습률을 감소시킬 스텝 크기, gamma: 학습률을 감소시킬 비율
step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)


train_losses = []  # Train loss 저장
train_accuracies = []  # Train accuracy 저장
val_losses = []  # Validation loss 저장
val_accuracies = []  # Validation accuracy 저장

# Early stopping을 위한 변수
best_val_accuracy = 0.0
patience = 10  # 검증 정확도가 patience 횟수 동안 개선되지 않으면 학습 종료
counter = 0

# Training loop
for epoch in range(30):  # 30 에폭 동안 학습합니다.
    model.train()
    train_loss = 0
    correct_train = 0
    total_train = 0

    for images, masks in tqdm(dataloader):
        images = images.float().to(device)
        masks = masks.long().to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks.squeeze(1))
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        # 정확도 계산
        _, predicted = torch.max(outputs, 1)
        total_train += masks.view(-1).size(0)
        correct_train += (predicted == masks.squeeze(1)).sum().item()

    train_accuracy = correct_train / total_train * 100  # 정확도를 100% 기준으로 계산
    train_losses.append(train_loss / len(dataloader))
    train_accuracies.append(train_accuracy)

    # Validation
    model.eval()
    val_loss = 0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for val_images, val_masks in tqdm(val_dataloader):
            val_images = val_images.float().to(device)
            val_masks = val_masks.long().to(device)

            val_outputs = model(val_images)
            val_loss += criterion(val_outputs, val_masks.squeeze(1)).item()

            # Validation 정확도 계산
            _, val_predicted = torch.max(val_outputs, 1)
            total_val += val_masks.view(-1).size(0)
            correct_val += (val_predicted == val_masks.squeeze(1)).sum().item()

    val_accuracy = correct_val / total_val * 100  # 정확도를 100% 기준으로 계산
    val_losses.append(val_loss / len(val_dataloader))
    val_accuracies.append(val_accuracy)

    step_lr_scheduler.step()

    print(f'Epoch {epoch+1}')
    print(f'Train Loss: {train_losses[-1]}, Train Accuracy: {train_accuracies[-1]:.2f}%')
    print(f'Validation Loss: {val_losses[-1]}, Validation Accuracy: {val_accuracies[-1]:.2f}%')

    if (epoch+1) % 5 == 0:
      # 그래프 그리기
      plt.figure(figsize=(12, 4))

      plt.subplot(1, 2, 2)
      plt.plot(range(1, len(train_accuracies) + 1), train_accuracies, marker='o', linestyle='-', label='Train Accuracy')
      plt.plot(range(1, len(val_accuracies) + 1), val_accuracies, marker='o', linestyle='-', label='Validation Accuracy')
      plt.xlabel('Epoch')
      plt.ylabel('Accuracy')
      plt.title('Training and Validation Accuracy')
      plt.grid(True)

      plt.show()

    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        counter = 0  # 현재 검증 정확도가 더 좋을 경우 counter를 리셋

        # 모델 저장
        best_model_path = os.path.join(save_dir, 'best_model.pth')
        torch.save(model.state_dict(), best_model_path)
        print(f'Saved the best model with validation accuracy: {best_val_accuracy:.2f} in {best_model_path}')

    else:
        counter += 1

    if counter >= patience:
        print(f'Early stopping at epoch {epoch+1} as validation accuracy did not improve for {patience} epochs.')
        break

# 모델 저장
        last_model_path = os.path.join(save_dir, 'last_model.pth')
        torch.save(model.state_dict(), last_model_path)
        print(f'Saved the last model with validation accuracy: {val_accuracy:.2f} in {last_model_path}')

# lr=0.001에서 최적

# 대략 loss 0.014차이가 score 0.017차이

## Inference

In [ ]:
test_dataset = CustomDataset(csv_file='./test.csv', transform=transform, infer=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

In [ ]:
# 저장한 모델의 경로 설정
model_path = './saved_models/best_model.pth' # 최적의 vaildation accuracy 모델을 사용하려는 경우
# model_path = './saved_models/last_model.pth' # 마지막에 학습된 모델을 사용하려는 경우

# 모델 초기화
model = LightweightPSPNet(in_channels, num_classes).to(device)

# 저장한 모델 파라미터 불러오기
model.load_state_dict(torch.load(model_path))

In [ ]:
with torch.no_grad():
    model.eval()
    result = []
    for images in tqdm(test_dataloader):
        images = images.float().to(device)
        outputs = model(images)
        outputs = torch.softmax(outputs, dim=1).cpu()
        outputs = torch.argmax(outputs, dim=1).numpy()
        # batch에 존재하는 각 이미지에 대해서 반복
        for pred in outputs:
            pred = pred.astype(np.uint8)
            pred = Image.fromarray(pred) # 이미지로 변환
            pred = pred.resize((960, 540), Image.NEAREST) # 960 x 540 사이즈로 변환
            pred = np.array(pred) # 다시 수치로 변환
            # class 0 ~ 11에 해당하는 경우에 마스크 형성 / 12(배경)는 제외하고 진행
            for class_id in range(12):
                class_mask = (pred == class_id).astype(np.uint8)
                if np.sum(class_mask) > 0: # 마스크가 존재하는 경우 encode
                    mask_rle = rle_encode(class_mask)
                    result.append(mask_rle)
                else: # 마스크가 존재하지 않는 경우 -1
                    result.append(-1)

## Submission

In [ ]:
submit = pd.read_csv('./sample_submission_1.csv')
submit['mask_rle'] = result
submit

In [ ]:
submit.to_csv('./sample_submission_1.csv', index=False)